In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#IDir = r'/Volumes/PhD/imagery/masters/ROC/M1_ROC.shp'
IDir = r'/Volumes/PhD/imagery/masters/output/ROC/design_storm/M1_design_storm.shp'
ODir = r'/Volumes/PhD/imagery/masters/hyper/'
dNBR_df = gpd.read_file(IDir)
dNBR_df

dnbr_basins = pd.read_csv(r'/Volumes/PhD/imagery/masters/hyper/basins.csv')
dnbr_basins

,BASIN_ID,Mean_dNBR,Med_dNBR,PF_dNBR,X1,X3
0,17346,0.562744,0.678311,0.786640,0.560555,0.131622
1,19384,0.631775,0.665531,0.701887,0.683425,0.139364
2,19451,0.638890,0.654705,0.655092,0.243529,0.130000
3,21224,0.365260,0.429648,0.682649,0.231744,0.203875


In [3]:
def M1(dnbr, Value, RI):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return {
        'BASIN_ID': basin_id,
        'RI': RI *4,
        'M1P': prob
    }
    

In [4]:
def HM1(dnbr, Value, RI):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['X1']
    x_3 = dnbr['X3']
    x_2 = dnbr[f'{Value}']
    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return {
        'BASIN_ID': basin_id,
        'RI': RI *4,
        'M1P': prob
    }
    

In [7]:
results = []
results_1 = []
results_2 = []
for basin_id in dnbr_basins['BASIN_ID'].unique():
    basin_data = dnbr_basins[dnbr_basins['BASIN_ID'] == basin_id].iloc[0]
    R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
    for RI in R:
        result = HM1(basin_data, 'Mean_dNBR', RI)
        results.append(result)
        M1P_Median = HM1(basin_data, 'Med_dNBR', RI)
        results_1.append(M1P_Median)
        M1P_PF = HM1(basin_data, 'PF_dNBR', RI)
        results_2.append(M1P_PF)


final_df = pd.DataFrame(results)
final_df_1 = pd.DataFrame(results_1)
final_df_2 = pd.DataFrame(results_2)


In [8]:
final_df.to_csv(os.path.join(ODir, 'HM1P_mean.csv'))
final_df_1.to_csv(os.path.join(ODir,'HM1P_med.csv'))
final_df_2.to_csv(os.path.join(ODir, 'HM1P_PF.csv'))

In [11]:
All_df = pd.DataFrame()
test = pd.DataFrame()

R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
for RI in R:
    M1P_Mean = M1(dNBR_df, 'Mean_dNBR')
    M1P_Median = M1(dNBR_df, 'Med_dNBR')
    M1P_PF = M1(dNBR_df, 'PF_dNBR')
    R_val = int(RI*4)
    testdf = {
        'RI': R_val,
        'M1P_M': M1P_Mean
    }
    testdf = pd.DataFrame(testdf)
    test = pd.concat([test, testdf], axis=1)
    M1P_df = {
        f'M1P_Mn_{R_val}': M1P_Mean,
        f'M1P_Md_{R_val}': M1P_Median,
        f'M1P_PF_{R_val}': M1P_PF
        }
    M1P_df = pd.DataFrame(M1P_df)   
    All_df = pd.concat([All_df, M1P_df], axis=1)

All_df




,M1P_Mn_12,M1P_Md_12,M1P_PF_12,M1P_Mn_16,M1P_Md_16,M1P_PF_16,M1P_Mn_20,M1P_Md_20,M1P_PF_20,M1P_Mn_24,...,M1P_PF_28,M1P_Mn_32,M1P_Md_32,M1P_PF_32,M1P_Mn_36,M1P_Md_36,M1P_PF_36,M1P_Mn_40,M1P_Md_40,M1P_PF_40
0,0.064164,0.056530,0.053738,0.086012,0.072894,0.068209,0.114390,0.093525,0.086222,0.150589,...,0.135529,0.250367,0.189059,0.168113,0.314323,0.234262,0.206657,0.386199,0.286454,0.251368
1,0.102490,0.084037,0.071389,0.156679,0.121858,0.098791,0.232110,0.173476,0.135180,0.329661,...,0.241169,0.565548,0.420700,0.311854,0.679271,0.523448,0.392540,0.775065,0.624249,0.479554
2,0.117073,0.107684,0.084992,0.184835,0.166659,0.123629,0.279405,0.248923,0.176443,0.398698,...,0.330730,0.659741,0.601332,0.428735,0.768286,0.714256,0.532669,0.850073,0.805538,0.633842
3,0.115671,0.102787,0.075314,0.182111,0.157248,0.105863,0.274854,0.233069,0.146836,0.392179,...,0.266688,0.651533,0.567658,0.345830,0.760925,0.681373,0.434541,0.844189,0.776932,0.527653
4,0.201483,0.209799,0.215912,0.348403,0.363968,0.375303,0.531189,0.552251,0.567236,0.705973,...,0.861859,0.915124,0.925092,0.931562,0.958067,0.963792,0.967424,0.979764,0.982868,0.984800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,0.060798,0.056792,0.052333,0.080175,0.073337,0.065876,0.105037,0.094220,0.082618,0.136466,...,0.128061,0.222709,0.191072,0.157935,0.278394,0.236910,0.193234,0.341878,0.289806,0.234228
1732,0.067893,0.067951,0.069137,0.092567,0.092670,0.094773,0.125006,0.125173,0.128602,0.166723,...,0.226756,0.281836,0.282330,0.292475,0.354677,0.355305,0.368171,0.434942,0.435691,0.450972
1733,0.081784,0.080229,0.077522,0.117692,0.114831,0.109877,0.166508,0.161732,0.153489,0.230285,...,0.281205,0.401568,0.388260,0.364939,0.501237,0.485577,0.457729,0.600808,0.583999,0.553547
1734,0.225089,0.219513,0.199548,0.392137,0.381935,0.344757,0.588938,0.575865,0.526175,0.760877,...,0.831838,0.940101,0.935070,0.912589,0.972111,0.969364,0.956588,0.987247,0.985820,0.978950


In [12]:


merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, All_df, left_index=True, right_index=True, how="left")
)

In [13]:
merged_gdf

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,M1P_PF_28,M1P_Mn_32,M1P_Md_32,M1P_PF_32,M1P_Mn_36,M1P_Md_36,M1P_PF_36,M1P_Mn_40,M1P_Md_40,M1P_PF_40
0,123,16060.0,6.322700e+06,123,1.0,10,0,0,thm2017,thm2017_123,...,0.135529,0.250367,0.189059,0.168113,0.314323,0.234262,0.206657,0.386199,0.286454,0.251368
1,155,3140.0,2.716000e+05,155,1.0,10,101,12,thm2017,thm2017_155,...,0.241169,0.565548,0.420700,0.311854,0.679271,0.523448,0.392540,0.775065,0.624249,0.479554
2,167,1800.0,7.260000e+04,167,1.0,10,100,11,thm2017,thm2017_167,...,0.330730,0.659741,0.601332,0.428735,0.768286,0.714256,0.532669,0.850073,0.805538,0.633842
3,172,2720.0,2.340000e+05,172,1.0,10,101,12,thm2017,thm2017_172,...,0.266688,0.651533,0.567658,0.345830,0.760925,0.681373,0.434541,0.844189,0.776932,0.527653
4,184,1540.0,4.900000e+04,184,1.0,10,100,11,thm2017,thm2017_184,...,0.861859,0.915124,0.925092,0.931562,0.958067,0.963792,0.967424,0.979764,0.982868,0.984800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,44218,2880.0,2.577000e+05,44218,1.0,10,100,11,thm2017,thm2017_44218,...,0.128061,0.222709,0.191072,0.157935,0.278394,0.236910,0.193234,0.341878,0.289806,0.234228
1732,44238,6260.0,8.933000e+05,44238,1.0,10,100,0,thm2017,thm2017_44238,...,0.226756,0.281836,0.282330,0.292475,0.354677,0.355305,0.368171,0.434942,0.435691,0.450972
1733,44256,1140.0,3.360000e+04,44256,1.0,10,100,11,thm2017,thm2017_44256,...,0.281205,0.401568,0.388260,0.364939,0.501237,0.485577,0.457729,0.600808,0.583999,0.553547
1734,44320,3020.0,1.722000e+05,44320,1.0,10,0,0,thm2017,thm2017_44320,...,0.831838,0.940101,0.935070,0.912589,0.972111,0.969364,0.956588,0.987247,0.985820,0.978950


In [15]:
# Step 1: Verify RI-related columns
ri_columns = [col for col in merged_gdf.columns if "MIP" in col]  # Adjust keyword as needed
print("RI Columns:", ri_columns)

# Step 2: Melt the DataFrame
long_df = pd.melt(
    merged_gdf,
    id_vars=["BASIN_ID"],  # Ensure BASIN_ID exists in the DataFrame
    value_vars=ri_columns,  # Use identified RI-related columns
    var_name="Variable",
    value_name="Value"
)
print("Melted DataFrame:")
print(long_df.head())

# Step 3: Extract RI values and variable types
long_df["RI"] = long_df["Variable"].str.extract(r"(\d+)$").astype(float)
long_df["Variable"] = long_df["Variable"].str.extract(r"^(.*)_R")

print("After Extracting RI and Variables:")
print(long_df.head())

# Step 4: Optional - Pivot the DataFrame
reshaped_df = long_df.pivot_table(
    index=["BASIN_ID", "RI"],  # Use BASIN_ID and RI as unique keys
    columns="Variable",  # Spread variables into separate columns
    values="Value"
).reset_index()

print("Reshaped DataFrame:")

RI Columns: []
Melted DataFrame:
Empty DataFrame
Columns: [BASIN_ID, Variable, Value]
Index: []
After Extracting RI and Variables:
Empty DataFrame
Columns: [BASIN_ID, Variable, Value, RI]
Index: []
Reshaped DataFrame:


In [28]:
merged_gdf.to_file(os.path.join(ODir, 'M1P_design_storm.shp'))